# Clonamos el repositorio con los modelos y herramientas

In [1]:
!git clone https://github.com/dannasalazar11/Msc_thesis.git

Cloning into 'Msc_thesis'...
remote: Enumerating objects: 268, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 268 (delta 61), reused 47 (delta 24), pack-reused 172 (from 1)
Receiving objects: 100% (268/268), 49.90 MiB | 34.60 MiB/s, done.
Resolving deltas: 100% (152/152), done.


In [2]:
import sys
sys.path.append('/kaggle/working/Msc_thesis')

from gmrrnet_adhd.utils import get_segmented_data, train_LOSO
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')


import tensorflow as tf
import numpy as np
import random
import os

# Establecer semilla
seed = 42

# Semillas para módulos principales
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

2025-04-30 15:14:30.271334: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746026070.704707      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746026070.825902      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Importar base de datos segmentada (Segmentos de 4 seg con translape del 50%, es decir, de 2 seg + el preprocesamiento del artículo)

In [3]:
import requests
import pickle
import io
from sklearn.preprocessing import OneHotEncoder
from gmrrnet_adhd.models.automatic_identification import automatic_identification

url = 'https://raw.githubusercontent.com/dannasalazar11/Msc_thesis/main/gmrrnet_adhd/L24SO/results/X_preprocessed.pkl'
response = requests.get(url)
X = pickle.load(io.BytesIO(response.content))

url = 'https://raw.githubusercontent.com/dannasalazar11/Msc_thesis/main/gmrrnet_adhd/L24SO/results/y.pkl'
response = requests.get(url)
y = pickle.load(io.BytesIO(response.content))
# y = y.reshape(-1, 1)  # sklearn espera 2D
# encoder = OneHotEncoder(sparse_output=False)
# y = encoder.fit_transform(y)

_, _, sbjs = get_segmented_data()

X.shape, y.shape, len(sbjs)

((8213, 836), (8213,), 8213)

# Importamos el modelo y definimos los hiperparámetros

In [4]:
model_name = 'Automatic_Identification'

X_pca, model = automatic_identification(X,y)

Selected features: 756
X shape after ANOVA filter: (8213, 756)
Original ANOVA-selected shape: (8213, 756)
Reduced shape after PCA: (8213, 177)
Number of components selected: 177


# Resultados - Leave One Subject Out

In [5]:
results = train_LOSO(model, X_pca, y, sbjs, None, None, 1, 120, model_name)

Evaluando modelo para el sujeto #1: v107
Métricas para v107: {'accuracy': 0.9868421052631579}

Evaluando modelo para el sujeto #2: v108
Métricas para v108: {'accuracy': 0.9863013698630136}

Evaluando modelo para el sujeto #3: v109
Métricas para v109: {'accuracy': 0.6721311475409836}

Evaluando modelo para el sujeto #4: v10p
Métricas para v10p: {'accuracy': 0.5740740740740741}

Evaluando modelo para el sujeto #5: v110
Métricas para v110: {'accuracy': 0.9682539682539683}

Evaluando modelo para el sujeto #6: v111
Métricas para v111: {'accuracy': 0.896551724137931}

Evaluando modelo para el sujeto #7: v112
Métricas para v112: {'accuracy': 1.0}

Evaluando modelo para el sujeto #8: v113
Métricas para v113: {'accuracy': 1.0}

Evaluando modelo para el sujeto #9: v114
Métricas para v114: {'accuracy': 1.0}

Evaluando modelo para el sujeto #10: v115
Métricas para v115: {'accuracy': 1.0}

Evaluando modelo para el sujeto #11: v116
Métricas para v116: {'accuracy': 1.0}

Evaluando modelo para el suje

In [6]:
results

{'v107': {'accuracy': 0.9868421052631579},
 'v108': {'accuracy': 0.9863013698630136},
 'v109': {'accuracy': 0.6721311475409836},
 'v10p': {'accuracy': 0.5740740740740741},
 'v110': {'accuracy': 0.9682539682539683},
 'v111': {'accuracy': 0.896551724137931},
 'v112': {'accuracy': 1.0},
 'v113': {'accuracy': 1.0},
 'v114': {'accuracy': 1.0},
 'v115': {'accuracy': 1.0},
 'v116': {'accuracy': 1.0},
 'v117': {'accuracy': 0.9797979797979798},
 'v118': {'accuracy': 1.0},
 'v120': {'accuracy': 1.0},
 'v121': {'accuracy': 0.9354838709677419},
 'v123': {'accuracy': 1.0},
 'v125': {'accuracy': 0.9491525423728814},
 'v127': {'accuracy': 0.75},
 'v129': {'accuracy': 1.0},
 'v12p': {'accuracy': 0.40298507462686567},
 'v131': {'accuracy': 0.984375},
 'v133': {'accuracy': 1.0},
 'v134': {'accuracy': 0.11764705882352941},
 'v138': {'accuracy': 0.7659574468085106},
 'v140': {'accuracy': 0.25757575757575757},
 'v143': {'accuracy': 0.4067796610169492},
 'v147': {'accuracy': 0.25925925925925924},
 'v149': {

In [7]:
import pickle

with open(f'results_LOSO_{model_name}.pkl', 'wb') as f:
    pickle.dump(results, f)